# Notebook version of 2.3_dps.py, which runs over a single tile
#### cant get 2.3_dps.py to work, so alternate approach is to loop over tile ids and return indiv tile CSVs of ATL08

In [1]:
#import pdal
import json
import os
import glob
import time

import itertools

import pandas as pd
from pyproj import CRS, Transformer

# From CovariateUtils
import numpy
from rasterio import enums
from rasterio.io import MemoryFile
from rasterio.crs import CRS
from rasterio.vrt import WarpedVRT
from rasterio.warp import array_bounds, calculate_default_transform
import geopandas

import folium
import shapely as shp

import argparse

from maap.maap import MAAP
maap = MAAP()

import sys
sys.path.append('/projects/code/icesat2_boreal/notebooks/3.Gridded_product_development')
if False:
    
    sys.path.append('/projects/code/icesat2_boreal/notebooks/2.ICESat-2_processing')

    #TODO: how to get this import right if its in a different dir
    from CovariateUtils import get_index_tile

import ExtractUtils
import FilterUtils



/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
maap_query = True

dps_dir_topo =       '/projects/r2d2/dps_output/do_topo_stack_3-1-5_ubuntu/master/2021/06/18'
dps_dir_csv =        '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17' #'/projects/jabba/dps_output/2.3_output'#
output_dir =         '/projects/jabba/data/out_tiles'
topo_stack_list_fn = '/projects/jabba/data/topo_stack_list.csv'
csv_list_fn =        '/projects/jabba/data/extract_atl08_csv_list.csv'
TEST = False
do_30m = True
extract_covars = True

in_tile_fn =         '/projects/maap-users/alexdevseed/boreal_tiles.gpkg' # '/projects/jabba/data/grid_boreal_albers_100k.gpkg'# 
in_tile_layer = 'boreal_tiles_albers'

thresh_h_can = 100
thresh_h_dif = 100
month_min = 6
month_max = 9

date_start = '06-01'
date_end = '09-30'
years_list = [2019, 2020, 2021]
# NA tiles
# Read the boreal tile index file

boreal_tile_index = geopandas.read_file(in_tile_fn)
boreal_tile_index_subset = boreal_tile_index.to_crs(4326).cx[-170:-50, 50:75]

# Boreal NA tiles: need just a list of tile_ids
INPUT_TILE_NUM_LIST = boreal_tile_index_subset['layer'].astype(int).tolist()

INPUT_TILE_NUM_LIST[0:5]


[3270, 3366, 3367, 3461, 3462]

#### Build the list of extracted CSV paths from DPS and save all paths in a list - glob.glob takes a while - dont do this for each tile

In [4]:
%%time
# This is sloooow
def get_atl08_csv_list(dps_dir_csv, seg_str, csv_list_fn):
    print(dps_dir_csv + "/**/ATL08*" + seg_str + ".csv") 
    #seg_str="_30m"
    all_atl08_csvs = glob.glob(dps_dir_csv + "/**/ATL08*" + seg_str + ".csv", recursive=True)
    print(len(all_atl08_csvs))
    all_atl08_csvs_df = pd.DataFrame({"path": all_atl08_csvs})
    all_atl08_csvs_df.to_csv(csv_list_fn)
    return(all_atl08_csvs_df)
    
def get_topo_stack_list(dps_dir_topo, topo_stack_list_fn):
    all_topo_stacks = glob.glob(dps_dir_topo + "/**/Copernicus_*_covars_cog_topo_stack.tif", recursive=True)
    print(len(all_topo_stacks))
    all_topo_stacks_df = pd.DataFrame({"path": all_topo_stacks})
    all_topo_stacks_df.to_csv(topo_stack_list_fn)
    return(all_topo_stacks_df)

def get_topo_stack_fn(topo_stack_list_fn, in_tile_num):
    # Find most recent topo stack path for tile in list of topo stack paths 
    all_topo_stacks_df = pd.read_csv(topo_stack_list_fn)
    stack_for_tile = all_topo_stacks_df[all_topo_stacks_df['path'].str.contains("Copernicus_"+str(in_tile_num))]
    #[print(i) for i in stack_for_tile.path.to_list()]
    topo_stack_fn = stack_for_tile.path.to_list()[0]
    return(topo_stack_fn)

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 9.3 µs


#### Check the CSVs produced from extract_atl08 for a given tile id

In [7]:
if os.path.isfile(csv_list_fn):

    all_atl08_for_tile = ExtractUtils.maap_search_get_h5_list(tile_num=14151, tile_fn=in_tile_fn, layer=in_tile_layer, DATE_START=date_start, DATE_END=date_end, YEARS=years_list)
    all_atl08_csvs_df = pd.read_csv(csv_list_fn)

    print([print(i) for i in all_atl08_for_tile])

    for g in all_atl08_for_tile:
        s = g.split("_")[1]
        f = all_atl08_csvs_df[all_atl08_csvs_df['path'].str.contains(s)]
        if len(f) > 0:
            print(f.path.iloc[0])
else:
    print(f'Need to create the csv list file: {csv_list_fn}')

Need to create the csv list file: /projects/jabba/data/extract_atl08_csv_list.csv


#### Check for duplicate CSVs

In [130]:
print(f'Length of CSV list from extract_atl08: {len(all_atl08_csvs_df)}')
just_basenames = [os.path.basename(all_atl08_csvs_df.path[i]) for i in range(0,len(all_atl08_csvs_df))]
no_dups = set(just_basenames)

print(f'Length of CSV list from extract_atl08 after removing duplicates: {len(no_dups)}')

# Check, and get list of duplicated ATL08 CSVs
import collections
dups = [item for item, count in collections.Counter(just_basenames).items() if count > 1]
print(f'# of duplicates in CSV: {len(dups)}')
len(dups)
dups[0:10]

Length of CSV list from extract_atl08: 10513
Length of CSV list from extract_atl08 after removing duplicates: 10119
# of duplicates in CSV: 394


['ATL08_30m_20190110100101_01980205_003_01_30m.csv',
 'ATL08_30m_20181108212000_06300106_003_01_30m.csv',
 'ATL08_30m_20181108210401_06300103_003_01_30m.csv',
 'ATL08_30m_20181107011951_06020106_003_01_30m.csv',
 'ATL08_30m_20181106215518_06000103_003_01_30m.csv',
 'ATL08_30m_20181111120611_06700105_003_01_30m.csv',
 'ATL08_30m_20181109001236_06320103_003_01_30m.csv',
 'ATL08_30m_20181113204035_07060105_003_01_30m.csv',
 'ATL08_30m_20181111084701_06680103_003_01_30m.csv',
 'ATL08_30m_20181103072045_05450102_003_01_30m.csv']

# Loop over input tiles to output ATL08 tile CSVs

In [8]:
%%time
INPUT_TILE_NUM_LIST = range(30823,30824) #, 30543, 30821, 30822, 30823]:
INPUT_TILE_NUM_LIST = [14151]#[30542]#[30543]
for in_tile_num in INPUT_TILE_NUM_LIST:
    
    # TODO: make this an arg
    years_list = [2019, 2020, 2021]
    
    seg_str = '_100m'
    if do_30m:
        seg_str = '_30m'
    if TEST:
        seg_str = '' 
    
    if maap_query and dps_dir_csv is not None:
        
        print("\nDoing MAAP query by tile bounds to find all intersecting ATL08 ")
        # Get a list of all ATL08 H5 granule names intersecting the tile (this will be a small list)
        all_atl08_for_tile = ExtractUtils.maap_search_get_h5_list(tile_num=in_tile_num, tile_fn=in_tile_fn, layer=in_tile_layer, DATE_START=date_start, DATE_END=date_end, YEARS=years_list)
        print([os.path.basename(f) for f in all_atl08_for_tile])      
        
        if not os.path.isfile(csv_list_fn):
            all_atl08_csvs_df = get_atl08_csv_list(dps_dir_csv, seg_str, csv_list_fn)
        else:
            print(f"\tReading existing list of ATL08 CSVs: {csv_list_fn}")
            all_atl08_csvs_df = pd.read_csv(csv_list_fn)
            
        # Find the ATL08 CSVs from extract that are associated with the ATL08 granules that intersect this tile
        # These CSvs are nested deep after DPS runs
        # They should match all the ATL08 granules, but probably wont bc: (1) did DPS for ATL08 30m fully complete with no fails? (2) did DPS for extract fully complete with no fails?
        all_atl08_csvs_FOUND, all_atl08_csvs_NOT_FOUND = FilterUtils.find_atl08_csv_tile(all_atl08_for_tile, all_atl08_csvs_df, seg_str) 
        
        #all_atl08_csvs_FOUND = [x for x in all_atl08_h5_with_csvs_for_tile if x not in all_atl08_csvs_NOT_FOUND]
        print("\t# of ATL08 CSV found for tile {}: {}".format(in_tile_num, len(all_atl08_csvs_FOUND)))
        print("\t# of ATL08 CSV NOT found for tile {}: {}".format(in_tile_num, len(all_atl08_csvs_NOT_FOUND)))
        if len(all_atl08_csvs_FOUND) == 0:
            print('\tNo ATL08 extracted for this tile.')
            continue
        
        # Merge all ATL08 CSV files for the current tile into a pandas df
        print("Creating pandas data frame...")
        atl08 = pd.concat([pd.read_csv(f) for f in all_atl08_csvs_FOUND ], sort=False, ignore_index=True)
        atl08 = FilterUtils.prep_filter_atl08_qual(atl08)

        print("\nFiltering by tile: {}".format(in_tile_num))
        # Get tile bounds as xmin,xmax,ymin,ymax
        tile = ExtractUtils.get_index_tile(in_tile_fn, in_tile_num, buffer=0, layer=in_tile_layer)
        in_bounds = FilterUtils.reorder_4326_bounds(tile)
        print(in_bounds)
        
        # Now filter ATL08 obs by tile bounds (changed to a geopandas clip instead of simple bounds approach)
        
        atl08 = FilterUtils.filter_atl08_bounds_clip(atl08, tile['geom_4326'])
        #atl08 = FilterUtils.filter_atl08_bounds(atl08_df=atl08, in_bounds=in_bounds)

    elif maap_query and dps_dir_csv is None:
        print("\nNo DPS dir specified: cant get ATL08 CSV list to match with tile bound results from MAAP query.\n")
        #os._exit(1)
        continue
    else:
        # Filter by bounds: EPT with a the bounds from an input tile
        atl08 = FilterUtils.filter_atl08_bounds_tile_ept(in_ept_fn, in_tile_fn, in_tile_num, in_tile_layer, output_dir, return_pdf=True)
    
    # Filter by quality
    atl08_pdf_filt = FilterUtils.filter_atl08_qual(atl08, SUBSET_COLS=True, DO_PREP=False,
                                                       subset_cols_list=['rh25','rh50','rh60','rh70','rh75','rh80','rh90','h_can','h_max_can'], 
                                                       filt_cols=['h_can','h_dif_ref','m','msw_flg','beam_type','seg_snow', 'seg_landcov'], 
                                                       thresh_h_can=100, thresh_h_dif=100, month_min=6, month_max=9)
    atl08=None
    
    # Convert to geopandas data frame in lat/lon
    atl08_gdf = geopandas.GeoDataFrame(atl08_pdf_filt, geometry=geopandas.points_from_xy(atl08_pdf_filt.lon, atl08_pdf_filt.lat), crs='epsg:4326')
    out_name_stem = "atl08_filt"
    atl08_pdf_filt=None
    
    if extract_covars and len(atl08_gdf) > 0:
        ### Below here should be re-worked to follow final chunk of nb 2.3 (6/15/2021)
        #
        # Extract topo covar values to ATL08 obs (doing a reproject to tile crs)
        # TODO: consider just running 3.1.5_dpy.py here to produce this topo stack right before extracting its values
        #topo_covar_fn = do_3_1_5_dp.main(in_tile_fn=in_tile_fn, in_tile_num=in_tile_num, tile_buffer_m=120, in_tile_layer=in_tile_layer, topo_tile_fn='https://maap-ops-dataset.s3.amazonaws.com/maap-users/alexdevseed/dem30m_tiles.geojson')
        topo_covar_fn = FilterUtils.get_topo_stack_fn(topo_stack_list_fn, in_tile_num)
        
        if topo_covar_fn is not None:
            atl08_gdf = ExtractUtils.extract_value_gdf(topo_covar_fn, atl08_gdf, ["elevation","slope","tsri","tpi", "slopemask"], reproject=True)
            out_name_stem = out_name_stem + "_topo"

        if False:
            # Extract landsat covar values to ATL08 obs
            # TODO: consider just running 3.1.2_dpy.py here
            landsat_covar_fn = do_3_1_2_dps.main(in_tile_fn=in_tile_fn, in_tile_num=in_tile_num, in_tile_layer=in_tile_layer, sat_api='https://landsatlook.usgs.gov/sat-api', local=args.local)
            atl08_gdf = ExtractUtils.extract_value_gdf(landsat_covar_fn, atl08_gdf, ['Blue', 'Green', 'Red', 'NIR', 'SWIR', 'NDVI', 'SAVI', 'MSAVI', 'NDMI', 'EVI', 'NBR', 'NBR2', 'TCB', 'TCG', 'TCW', 'ValidMask', 'Xgeo', 'Ygeo'], reproject=False)
            out_name_stem = out_name_stem + "_landsat"
        
    if len(atl08_gdf) == 0:
        print(f"No ATL08 obs. for tile {in_tile_num}")
    else:
        # CSV the file
        cur_time = time.strftime("%Y%m%d") #"%Y%m%d%H%M%S"
        out_fn = os.path.join(output_dir, out_name_stem + "_" + cur_time + "_" + str(in_tile_num) )
        atl08_gdf.to_csv(out_fn+".csv", index=False, encoding="utf-8-sig")
        atl08_gdf.to_file(out_fn+'.geojson', driver="GeoJSON")

        print("Wrote output csv/geojson of filtered ATL08 obs with topo and Landsat covariates for tile {}: {}".format(in_tile_num, out_fn) )


Doing MAAP query by tile bounds to find all intersecting ATL08 
	TILE_NUM: 14151 (90.5908057715438,69.20064754680081,91.51877055347914,69.52624448724598)
		# ATL08 for tile 14151: 38
['ATL08_20190617100158_12240305_003_01.h5', 'ATL08_20190620210844_12770303_003_01.h5', 'ATL08_20190621095338_12850305_003_01.h5', 'ATL08_20190624210024_13380303_003_01.h5', 'ATL08_20190716083750_02790405_003_01.h5', 'ATL08_20190719194438_03320403_003_01.h5', 'ATL08_20190720082932_03400405_003_01.h5', 'ATL08_20190723193622_03930403_003_01.h5', 'ATL08_20190724082115_04010405_003_01.h5', 'ATL08_20190727192804_04540403_003_01.h5', 'ATL08_20190818070544_07820405_003_01.h5', 'ATL08_20190821181232_08350403_003_01.h5', 'ATL08_20190822065725_08430405_003_01.h5', 'ATL08_20190825180413_08960403_003_01.h5', 'ATL08_20190916054149_12240405_003_01.h5', 'ATL08_20190919164836_12770403_003_01.h5', 'ATL08_20190920053329_12850405_003_01.h5', 'ATL08_20190923164018_13380403_003_01.h5', 'ATL08_20190927163159_00120503_003_01.h5'

# Map checks

In [18]:
atl08_gdf.info()
boreal_tile_index_orig = geopandas.read_file('/projects/maap-users/alexdevseed/boreal_tiles.gpkg')

#tile_parts = ExtractUtils.get_index_tile(in_tile_fn, in_tile_num, buffer=0, layer=in_tile_layer)
#atl08_filt_test = pd.read_csv('/projects/jabba/data/atl08_filt_30543_topo_landsat_20210608185220.csv')
#atl08_filt_test_gdf = geopandas.GeoDataFrame(atl08_filt_test, crs='epsg:4326', geometry=geopandas.points_from_xy(atl08_filt_test.lon, atl08_filt_test.lat))

# Reformat bbox
bbox_list = tile["bbox_4326"]
print(bbox_list)
center = tile["geom_orig"].centroid.to_crs(4326)

m = folium.Map(
    # Zoom to center
    location=[center.y,center.x],
    # Zoom to corner
    #location=[bbox_list[1], bbox_list[2]],
    tiles="cartodbpositron",
    zoom_start=11,
)

geom_style = {'fillColor': 'green', 'color': 'green'}
bbox_style = {'fillColor': '#ff0000', 'color': '#ff0000'}
bbox_buf_style = {'fillColor': '#fdae61', 'color': '#fdae61'}

#folium.GeoJson(boreal_tile_index_orig, name='tiles').add_to(m)

folium.GeoJson(atl08_gdf, name="ATL08").add_to(m)
#folium.GeoJson(atl08_filt_test_gdf, name="ATL08").add_to(m)
folium.GeoJson(shp.geometry.box(*bbox_list), 
               name="bbox",
               style_function=lambda x:bbox_style).add_to(m)
folium.GeoJson(tile['geom_4326'], name='tile', style_function=lambda x:geom_style).add_to(m)
m

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   lon        0 non-null      float64 
 1   lat        0 non-null      float64 
 2   rh25       0 non-null      float64 
 3   rh50       0 non-null      float64 
 4   rh60       0 non-null      float64 
 5   rh70       0 non-null      float64 
 6   rh75       0 non-null      float64 
 7   rh80       0 non-null      float64 
 8   rh90       0 non-null      float64 
 9   h_can      0 non-null      float64 
 10  h_max_can  0 non-null      float64 
 11  geometry   0 non-null      geometry
dtypes: float64(11), geometry(1)
memory usage: 0.0 bytes
[83.28732668968276, 43.77415885922252, 83.70052750715054, 44.08299617564999]


# Debugging...

In [7]:
if False:
    import pandas as pd
    import FilterUtils

    all_atl08_csvs_FOUND = ['/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/22/56/24/607120/ATL08_30m_20190621110851_12860302_003_01_30m.csv', 
                            '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/12/32/945664/ATL08_30m_20190818082057_07830402_003_01_30m.csv', 
                            '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/11/17/197105/ATL08_30m_20190822081238_08440402_003_01_30m.csv']
    atl08_tmp = pd.concat([pd.read_csv(f) for f in all_atl08_csvs_FOUND ], sort=False, ignore_index=True)
    in_bounds = [-117.10749852280769, -116.50936927974429, 50.78795362739066, 51.16389512140189]
    atl08_tmp = atl08_tmp[(atl08_tmp.lat > in_bounds[2]) &
                           (atl08_tmp.lat < in_bounds[3]) &
                           (atl08_tmp.lon > in_bounds[0]) &
                           (atl08_tmp.lon < in_bounds[1])
                                   ]
    
    print('DEBUG FilterUtils.prep_filter_atl08_qual')
    atl08_tmp = FilterUtils.prep_filter_atl08_qual(atl08_tmp)
    print(len(atl08_tmp))
  